## Isogeny skips

This notebook contains a simple visualizations for data collected by glitching CSIDH on Chipwhisperer, goal is to check how often do glitches cause a skip of single isogeny.

In [ ]:
%run ./init.ipynb # loads datasets, etc.

In [ ]:
from csidh import CSIDHDLL

In [ ]:
SRC_PATH = "../src"
csidh = CSIDHDLL(SRC_PATH)

In [ ]:
# Since keyspace is small, we can just precompute everything
keyspace = {}
for e_1 in range(-10, 11):
    for e_2 in range(-10, 11):
        for e_3 in range(-10, 11):
            key = [e_1, e_2, e_3]
            csidh.public = 0xec  # Started at public key corresponding to [-10, 10, -10] = 144 * 409 % 419 = 0xec
            csidh.private = key
            res = csidh.action()
            keyspace[str(key)] = csidh.from_projective(res)

In [ ]:
keyspace[str([10, -10, 10])]

In [ ]:
def skip_isogeny(private, i):
    from copy import copy
    new = copy(private)
    if new[i] == 0:
        return new
    new[i] = new[i] - 1 if new[i] > 0 else new[i] + 1
    return new

# Attack 1
- First attack was randomly glitching whole group action evaluation

## First dataset - key = [10,-10 10]
- Key was [10, -10, 10], we started at public corresponding to [-10, 10, -10], that is 0xec in projective coordinates
- First, check how many of results correspond to one isogeny skip, then try to classify other types of faults

In [ ]:
private = [10, -10, 10]
expected_public = 0
df = datasets["dummy"][str(private)]["attack2-tight"]

statistics = {}
statistics["total"] = len(df)
statistics["mutes"] = len(df[df["good/bad/crash"] == "crash"])
statistics["good public"] = len(df[df["good/bad/crash"] == "good"])
statistics["bad public"] = len(df[df["good/bad/crash"] == "bad"])

statistics["skipped 3-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 0))]])
statistics["skipped 5-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 1))]])
statistics["skipped 7-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 2))]])

df = df.dropna()
df["possible"] = df["public"].isin(keyspace.values())
statistics["impossible public"] = len(df[df["possible"] == False])

In [ ]:
df[df["public"] == keyspace[str(skip_isogeny(private, 2))]]

In [ ]:
import seaborn as sns


i3skip = df[df["public"] == keyspace[str(skip_isogeny(private, 0))]]
i5skip = df[df["public"] == keyspace[str(skip_isogeny(private, 1))]]
i7skip = df[df["public"] == keyspace[str(skip_isogeny(private, 2))]]

data = {
    'time': list(i3skip["scope.glitch.ext_offset"]) + list(i5skip["scope.glitch.ext_offset"]) + list(i7skip["scope.glitch.ext_offset"]),
    'y': len(i3skip) * [1] + len(i5skip) * [0] + len(i7skip) * [-1],
    'type': len(i3skip) * ['3-isogeny skip'] + len(i5skip) * ['5-isogeny skip'] + len(i7skip) * ['7-isogeny skip']
}

df = pd.DataFrame(data)
sns.scatterplot(data=df, x='time', y='y', hue='type', style='type', s=100)

In [ ]:
sns.histplot(data=df, x='time', stat="count", multiple="stack"
             , kde=False, palette="pastel", element="bars", hue='type', bins=5)

In [ ]:
statistics

## Second dataset - key = [10, 0 0]



In [ ]:
private = [10, 0, 0]
expected_public = 295
df = datasets["dummy"][str(private)]["attack1"]
df

statistics = {}
statistics["total"] = len(df)
statistics["mutes"] = len(df[df["good/bad/crash"] == "crash"])
statistics["good public"] = len(df[df["good/bad/crash"] == "good"])
statistics["bad public"] = len(df[df["good/bad/crash"] == "bad"])

statistics["skipped 3-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 0))]])
#statistics["skipped 5-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 1))]])
#statistics["skipped 7-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 2))]])

df = df.dropna()
df["possible"] = df["public"].isin(keyspace.values())
statistics["impossible public"] = len(df[df["possible"] == False])

In [ ]:
statistics

In [ ]:
import seaborn as sns


i3skip = df[df["public"] == keyspace[str(skip_isogeny(private, 0))]]
#i5skip = df[df["public"] == keyspace[str(skip_isogeny(private, 1))]]
#i7skip = df[df["public"] == keyspace[str(skip_isogeny(private, 2))]]

data = {
    'time': list(i3skip["scope.glitch.ext_offset"]), #+ list(i5skip["scope.glitch.ext_offset"]) + list(i7skip["scope.glitch.ext_offset"]),
    'y': len(i3skip) * [1], #+ len(i5skip) * [0] + len(i7skip) * [-1],
    'type': len(i3skip) * ['3-isogeny skip'],# + len(i5skip) * ['5-isogeny skip'] + len(i7skip) * ['7-isogeny skip']
}

df = pd.DataFrame(data)
sns.scatterplot(data=df, x='time', y='y', hue='type', style='type', s=100)

In [ ]:
sns.histplot(data=df, x='time', bins=5)

# Attack 2

In [ ]:
private = [10, -10, 10]
df = datasets["dummy"][str(private)]["attack2"]
df

statistics = {}
statistics["total"] = len(df)
statistics["mutes"] = len(df[df["good/bad/crash"] == "crash"])
statistics["good public"] = len(df[df["good/bad/crash"] == "good"])
statistics["bad public"] = len(df[df["good/bad/crash"] == "bad"])

statistics["skipped 3-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 0))]])
statistics["skipped 5-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 1))]])
statistics["skipped 7-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 2))]])

df = df.dropna()
df["possible"] = df["public"].isin(keyspace.values())
statistics["impossible public"] = len(df[df["possible"] == False])

In [ ]:
statistics

In [ ]:
private = [10, -10, 0]
df = datasets["dummy"][str(private)]["attack2"]
df

statistics = {}
statistics["total"] = len(df)
statistics["mutes"] = len(df[df["good/bad/crash"] == "crash"])
statistics["good public"] = len(df[df["good/bad/crash"] == "good"])
statistics["bad public"] = len(df[df["good/bad/crash"] == "bad"])

statistics["skipped 3-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 0))]])
statistics["skipped 5-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 1))]])
#statistics["skipped 7-isogeny"] = len(df[df["public"] == keyspace[str(skip_isogeny(private, 2))]])

df = df.dropna()
df["possible"] = df["public"].isin(keyspace.values())
statistics["impossible public"] = len(df[df["possible"] == False])

In [ ]:
statistics